<div class='alert alert-info'>
    <h1>Método de máxima verosimilitud para la regresión logística</h1>

</div>

In [1]:
from IPython.display import display, Math

In [52]:
# Función de entorno
display(Math(r'L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-p_i)^{y_i}'))
def likelihood(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in = np.where(y[i]==1, pi[i], 1-pi[i])
        total_sum *= sum_in
    return total_sum
        

<IPython.core.display.Math object>

In [47]:
# Probabilidades para cada observación
display(Math(r'P_i = \frac{1}{1 + e^{\sum_{j=0}^k -\beta_j\cdot x_{ij}}}'))
def logitprobs(x, beta):
    import numpy as np
    n_row = np.shape(x)[0]
    n_col = np.shape(x)[1]
    pi = list(range(1, n_row+1))
    for i in range(n_row):
        expo = 0
        for j in range(n_col):
            expo += x[i][j] * beta[j]
        with np.errstate(divide='ignore', invalid='ignore'):
            pi[i] = 1 / (1 + np.exp(-expo))
    return pi

<IPython.core.display.Math object>

In [29]:
# Matriz diagonal W
display(Math(r'W = diag(P_i\cdot(1 - P_i))_{i-1}^n'))
def findW(pi):
    import numpy as np
    n = len(pi)
    w = np.zeros((n, n))
    for i in range(n):
        w[i, i] = float(pi[i]*(1 - pi[i]))
    return w

<IPython.core.display.Math object>

In [35]:
# Función logística
display(Math(r'\beta_{n+1} = \beta_n - \frac{f(\beta_n)}{f´(\beta_n)}'))
display(Math(r'f(\beta) = \beta(y-P)'))
display(Math(r'f´(\beta) = \beta W \beta^T'))

def logistic(x, y, limit):
    """
    Limit_ Limite del cambio de un paso al siguiente
    """
    import numpy as np
    n_row = np.shape(x)[0]
    bias = np.ones((n_row, 1))
    x_new = np.append(x, bias, axis=1)
    n_col = np.shape(x_new)[1]
    beta = np.zeros((n_col, 1))
    iter_i = 10000
    while(iter_i > limit):
        pi = logitprobs(x_new, beta)
        w = findW(pi)
        # deben ser vectores columna
        num = np.transpose(np.matrix(x_new)) * np.matrix(y - np.transpose(pi)).transpose()
        den = np.matrix(np.transpose(x_new) * np.matrix(w) * np.matrix(x_new))
        root_dif = np.array(np.linalg.inv(den) * num)
        beta += root_dif
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(y, pi)
    return beta

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Comprobemos

In [39]:
import numpy as np

In [40]:
x = np.array(range(10)).reshape(10,1)
y = [0, 0, 0, 0, 1, 0, 1, 0, 1, 1]
bias = np.ones((10, 1))
x_new = np.append(x, bias, axis=1)
x_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [53]:
a = logistic(x, y, 0.001)

In [54]:
a

array([[ 0.66217766],
       [-3.6953843 ]])

### y = -3.69 + x*0.66

# Verifiquemos que respultado nos daria python

In [58]:
import statsmodels.api as sm

In [63]:
model = sm.Logit(y, x_new);
result = model.fit();
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6
                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.360   
Dependent Variable: y                AIC:              12.6202 
Date:               2019-11-15 13:32 BIC:              13.2254 
No. Observations:   10               Log-Likelihood:   -4.3101 
Df Model:           1                LL-Null:          -6.7301 
Df Residuals:       8                LLR p-value:      0.027807
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const    -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906

